# Додавання необхідних бібліотек

In [34]:
import numpy as np

# Теоретичні Вказівки

Процес Маркова прийняття рішень $(MDP)$ представляє систему, яка визначається станами $S$, діями $A$, ймовірностями переходу між станами $P_{s,a}$, фактором дисконтування, або discount factor -- $γ$, і функцією винагороди $R$.

Ціль полягає у максимізації очікуваної винагороди, керуючи процесом через вибір оптимальної послідовності дій або політики $π$.

Алгоритми для пошуку оптимальної політики включають ітерацію політики (Алгоритм 5) та навчання моделі для невідомих MDP (Алгоритм 6).


## Алгоритм 5: Ітерація політики

Цей алгоритм використовується для знаходження оптимальної політики у процесі Маркова прийняття рішень $(MDP)$.
Він поетапно оновлює політику та функцію вартості для кожного стану, поки не досягне оптимального результату або поки не завершаться ітерації.
Алгоритм включає два основних кроки: оцінку політики та поліпшення політики.
Оцінка політики обчислює значення функції вартості для поточної політики, а поліпшення оновлює політику для максимізації очікуваної винагороди.

### Параметри:

1. $states$: Список станів у системі (наприклад, [0, 1, 2, 3]).

2. $actions$: Список можливих дій, які можна виконувати у кожному стані (наприклад, [0, 1]).

3. $transition\_prob$: Ймовірності переходу між станами для кожної дії (наприклад, вкладений словник, що містить ймовірності переходів між станами для кожної дії).

4. $rewards$: Список винагород для кожного стану.

5. $gamma$ (опціонально): Фактор дисконтування, який визначає значущість майбутніх винагород (значення за замовчуванням — 0.9).

6. $max\_iterations$ (опціонально): Максимальна кількість ітерацій для збіжності алгоритму (значення за замовчуванням — 1000).

7. $tol$ (опціонально): Допуск для збіжності значень функції вартості (значення за замовчуванням — 1e-6).

In [35]:
def policy_iteration(states, actions, transition_prob, rewards, gamma=0.9, max_iterations=1000, tol=1e-6):
    policy = np.random.choice(actions, size=len(states))
    value_function = np.zeros(len(states))

    is_converged = False
    iteration = 0
    while not is_converged and iteration < max_iterations:
        # Крок a: Оцінка Політики
        # На цьому кроці обчислюємо значення функції вартості для поточної політики, доки зміни значень не стануть менше tol.
        while True:
            delta = 0
            for s in range(len(states)):
                v = value_function[s]
                action = policy[s]
                value_function[s] = rewards[s] + gamma * sum(
                    transition_prob[s][action][s_next] * value_function[s_next] for s_next in range(len(states))
                )
                delta = max(delta, abs(v - value_function[s]))
            if delta < tol:
                break

        # Крок b: Поліпшення Політики
        # На цьому кроці для кожного стану обираємо дію, яка максимізує очікувану винагороду, використовуючи поточну функцію вартості.
        policy_stable = True
        for s in range(len(states)):
            old_action = policy[s]
            policy[s] = max(actions, key=lambda a: sum(
                transition_prob[s][a][s_next] * value_function[s_next] for s_next in range(len(states))
            ))
            if old_action != policy[s]:
                policy_stable = False

        if policy_stable:
            is_converged = True
        iteration += 1

    return policy, value_function




## Алгоритм 6: Навчання моделі MDP з невідомими ймовірностями переходу

Цей алгоритм використовується для навчання процесу Маркова прийняття рішень, коли ймовірності переходу між станами невідомі.
Алгоритм включає збір даних про переходи та винагороди під час виконання поточної політики, оцінку ймовірностей переходу, та оновлення політики.
Алгоритм працює у кілька ітерацій: виконується поточна політика, оновлюються оцінки ймовірностей та визначається нова оптимальна політика.

### Параметри:

1. $states$: Список станів у системі (наприклад, [0, 1, 2, 3]).

2. $actions$: Список можливих дій, які можна виконувати у кожному стані (наприклад, [0, 1]).

3. $transition\_prob$: Ймовірності переходу між станами для кожної дії (наприклад, вкладений словник, що містить ймовірності переходів між станами для кожної дії).

4. $rewards$: Список винагород для кожного стану.

5. $gamma$ (опціонально): Фактор дисконтування, який визначає значущість майбутніх винагород (значення за замовчуванням — 0.9).

6. $max\_iterations$ (опціонально): Максимальна кількість ітерацій для збіжності алгоритму (значення за замовчуванням — 1000).

7. $tol$ (опціонально): Допуск для збіжності значень функції вартості (значення за замовчуванням — 1e-6).



In [36]:
def learn_mdp_with_unknown_transitions(states, actions, transition_prob, rewards, gamma=0.9, max_iterations=1000, tol=1e-6):
    policy = np.random.choice(actions, size=len(states))
    transition_counts = {s: {a: np.zeros(len(states)) for a in actions} for s in states}

    is_converged = False
    iteration = 0
    while not is_converged and iteration < max_iterations:
        # Крок a: Виконання політики у MDP протягом деякої кількості спроб
        # Виконуємо поточну політику, щоб зібрати дані про переходи між станами та винагороди
        for _ in range(200):  # Збільшена кількість спроб для кращого збору даних
            s = np.random.choice(states)
            while True:
                a = policy[s]
                # Нормалізувати ймовірності переходів, щоб їх сума дорівнювала 1
                if np.sum(transition_prob[s][a]) == 0:
                    probs = np.ones(len(states)) / len(states)  # Якщо сума ймовірностей дорівнює 0, використовуємо рівномірний розподіл
                else:
                    probs = transition_prob[s][a] / np.sum(transition_prob[s][a])
                s_next = np.random.choice(states, p=probs)  # Симуляція переходу на основі ймовірностей
                reward = rewards[s]  # Симуляція винагороди (може бути замінена реальними даними на практиці)
                transition_counts[s][a][s_next] += 1
                s = s_next
                if np.random.rand() < 0.2:  # Збільшена ймовірність завершення спроби для більшого варіювання
                    break

        # Крок b: Оновлення оцінок ймовірностей переходів
        # Обчислюємо ймовірності переходів на основі зібраних даних (частота переходів)
        transition_prob = {
            s: {
                a: transition_counts[s][a] / (np.sum(transition_counts[s][a]) + 1e-6)
                for a in actions
            }
            for s in states
        }

        # Крок c: Оновлення політики за допомогою Алгоритму 5
        # Використовуємо ітерацію політики, щоб оновити політику на основі нових оцінених ймовірностей переходу
        policy, _ = policy_iteration(states, actions, transition_prob, rewards, gamma, max_iterations, tol)
        iteration += 1

    return policy


## Приклад використання

In [37]:
states = [0, 1, 2, 3]
actions = [0, 1]  # Приклад дій (наприклад, 0: залишитися, 1: рухатися)
transition_prob = {
    0: {0: [0.3, 0.7, 0, 0], 1: [0.2, 0.3, 0.5, 0]},
    1: {0: [0.4, 0.6, 0, 0], 1: [0.1, 0.6, 0.3, 0]},
    2: {0: [0.1, 0.5, 0.4, 0], 1: [0.2, 0.4, 0.4, 0]},
    3: {0: [0.3, 0.3, 0.4, 0], 1: [0.4, 0.4, 0, 0.2]},
}
rewards = [2, -1, 5, 3]

# Запуск ітерації політики з різними значеннями gamma для оцінки впливу на результати
gamma_values = [0.5, 0.9, 0.99]
for gamma in gamma_values:
    optimal_policy, optimal_value_function = policy_iteration(states, actions, transition_prob, rewards, gamma=gamma)
    print(f"\nGamma: {gamma}")
    print("Optimal Policy:", optimal_policy)
    print("Optimal Value Function:", optimal_value_function)

# Запуск алгоритму навчання з різною кількістю спроб і значеннями gamma
different_trials = [50, 100, 200]
for trials in different_trials:
    learned_policy = learn_mdp_with_unknown_transitions(states, actions, transition_prob, rewards, gamma=0.9, max_iterations=trials)
    print(f"\nNumber of Trials: {trials}")
    print("Learned Policy:", learned_policy)



Gamma: 0.5
Optimal Policy: [1 1 1 0]
Optimal Value Function: [4.18384372 0.33983256 6.85793854 5.05013915]

Gamma: 0.9
Optimal Policy: [1 1 1 0]
Optimal Value Function: [17.97141876 13.2787226  20.33624248 18.75858546]

Gamma: 0.99
Optimal Policy: [1 1 1 0]
Optimal Value Function: [167.3456328  162.43398811 169.63293741 168.11919063]

Number of Trials: 50
Learned Policy: [0 1 1 0]

Number of Trials: 100
Learned Policy: [0 0 1 0]

Number of Trials: 200
Learned Policy: [0 0 0 1]
